# Header

Starting from 0xC800, there seems to be a 0x200 size header block for each animation

    ---------------------------------------------------------------------
    0001 02 03 0405060708 09 0A 0B 0C 0D 0E 0F 10 11 12 13 <string name> <0x00 padding>
    ---------------------------------------------------------------------
    0001 01 58 000061A870 57 20 02 01 01 01 2E 03 2F 3B 01 24_001              0x0012C400
    0002 01 2C 0000630938 57 20 02 01 01 01 5A 0A 01 2C 01 24_002              0x0013EE00
    0003 01 06 000063BA18 29 20 01 00 00 00 00 00 00 00 00 24_003              0x00143000
    0004 01 0A 000063D31E 22 20 01 00 00 00 00 00 00 00 00 24_004              0x00148800
    0005 01 05 000063FC1E 83 20 01 00 00 00 00 00 00 00 01 24_005              0x00151A00
    0006 01 29 0000641129 57 20 02 01 01 01 05 0A 01 29 01 24_006
    0007 01 08 000064B630 AE 20 01 00 00 00 00 00 00 00 01 24_007
    0008 01 1D 000064D71D 17 1D 02 01 01 01 03 0A 01 23 01 24_008
    0009 01 3F 0000654C50 57 20 02 01 01 01 03 0A 01 3F 00 24_009
    000A 01 10 0000664910 80 20 01 01 01 00 00 00 00 00 00 AC#DC_001           0x00172C00
    000B 01 20 0000668A20 80 20 01 01 00 00 00 00 00 00 00 AC#DC_002
    022E 01 01 0000FFFD01 80 20 01 00 00 00 00 00 00 00 00 FLINTSTONES_024
    022F 01 01 0001000201 80 20 01 00 00 00 00 00 00 00 00 FLINTSTONES_025
    0230 01 01 0001000701 80 20 01 00 00 00 00 00 00 00 00 FLINTSTONES_026
    08FC 01 23 00034AB723 80 20 01 01 00 00 00 00 00 00 00 X-MEN_063
    08FD 01 14 00034B4414 80 20 01 00 00 00 00 00 00 00 00 X-MEN_064
    08FE 01 39 00034B9539 80 20 00 01 01 00 00 00 00 00 00 X-MEN_065


        0001 : Animation ID
          02 : Always 1
          03 : Animation length [frames]
    04050607 : Some kind of offset ??
          09 : xResolution [pixels]
          0A : yResolution [pixels]

# Animation offsets
    
    0x12C400: Start of first frame of first animation
    0x13EE00: Start of first frame of second animation
    0x143000: Start of first frame of third animation
    0x148800
    




__Strange data blocks at the end of an animation__
    
    0013EC00   01 32 02 32  03 32 04 32  05 32 06 32  07 32 08 32  .2.2.2.2.2.2.2.2
    0013EC10   01 32 02 32  03 32 04 32  05 32 06 32  07 32 08 32  .2.2.2.2.2.2.2.2
    0013EC20   01 32 02 32  03 32 04 32  05 32 06 32  07 32 08 32  .2.2.2.2.2.2.2.2
    0013EC30   01 32 02 32  03 32 04 32  05 32 06 32  07 32 08 32  .2.2.2.2.2.2.2.2
    0013EC40   01 32 02 32  03 32 04 32  05 32 06 32  07 32 08 32  .2.2.2.2.2.2.2.2
    0013EC50   01 32 02 32  03 32 04 32  05 32 06 32  07 32 08 32  .2.2.2.2.2.2.2.2
    
    00142E00   01 69 02 32  03 32 04 32  05 32 06 32  07 32 08 32  .i.2.2.2.2.2.2.2
    00142E10   09 32 0A 32  0B 6E 0C 69  0D 32 0E 32  0F 32 10 32  .2.2.n.i.2.2.2.2
    00142E20   11 32 12 32  13 32 14 32  15 69 16 32  17 32 18 32  .2.2.2.2.i.2.2.2
    00142E30   19 32 1A 32  1B 32 1C 32  1D 73 00 00  00 00 00 00  .2.2.2.2.s......


In [984]:
0x1D

29

In [11]:
%pylab
import time
import matplotlib.animation as animation

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [13]:
#fName = "./ani.raw.data"
fName = "/media/sf_Google_Drive/pdmd/picodmd_b236.img"
with open(fName,"rb") as f:
    rawDat = array( bytearray(f.read()), dtype=uint8 )
a = zeros((2*len(rawDat)), dtype=uint8)
# unpack 2 pixels / byte to 1 pixel / byte
a[1::2] = rawDat & 0x0F
a[0::2] = rawDat >> 4

In [511]:
close("all")
figure( figsize=(8,2))
d = imshow( zeros((32,128)), aspect="equal", vmin=0, vmax=15, interpolation="hanning", cmap=cm.binary_r )

xSize = 128
ySize = 32   #len(rawDat)
frameSize = xSize * ySize
frame = 700
def ani(x):
    global frame
    offset = xSize*ySize//2*frame + xSize*16
    b = array( a[offset:offset+ySize*xSize] ).reshape((-1,xSize))
    d.set_data( b )
    frame += 1
#ani = animation.FuncAnimation(gcf(), ani, interval=200 )

In [512]:
# Print a pretty hex-dump on the debug out
def hexDump( buffer ):
    for i,b in enumerate(buffer):
        if len(buffer)>16 and (i%16)==0:
            print( "\n    {:04x}: ".format(i), end='' )
        print( "{:02x} ".format(b), end='' )

In [513]:
def getFrame( buf, xSize=128, ySize=32, frameOffset=0, byteOffset=None ):
    if byteOffset is None:
        byteOffset = xSize*ySize*frameOffset       
    print( hex(byteOffset//2) )
    return a[byteOffset:byteOffset+ySize*xSize].reshape((-1,xSize))

In [945]:
128-41

87

In [944]:
d.set_data( getFrame(a, byteOffset=0x151A00*2 + frameSize*62 ) )

0x170a00


In [941]:
0x151A00

1382912

In [946]:
i=0
startOffset = 1345536

In [982]:
d.set_data( getFrame(a, byteOffset=startOffset*2 + frameSize*i ) )
print(i)
i += 1

0x15a000
35


In [230]:
getFrame(a, byteOffset=0x12c400*2 + frameSize*0 )[14]

0x12c400


array([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10,  0,  0,  0,  0,  0,  0,  0,  7,  7,  7,
        2,  2,  2,  2,  0,  4,  6,  6,  6,  6,  4,  3,  0,  4,  4,  4,  4,
        4,  2,  0,  0,  2,  1,  0,  3,  3,  0,  1,  0,  0,  0,  0,  0,  0,
        0,  1,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  4,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=uint8)